In [ ]:
!pip install pinecone --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.8/419.8 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
#Import Necessary Libraries

import numpy as np
import pandas as pd
import ast
import torch
from tqdm import tqdm
import pinecone
import multiprocessing as mp
from transformers import CLIPProcessor, CLIPModel, CLIPVisionModelWithProjection, CLIPTextModelWithProjection
from PIL import Image

In [ ]:
import requests



#Identify Blank and Non-Blank Images
img = Image.new('RGB', (200, 200), color='white')
# model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
model = CLIPVisionModelWithProjection.from_pretrained("openai/clip-vit-base-patch16")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")
model.eval()
with torch.no_grad():
  image_inputs = processor(images=img, return_tensors="pt")
  # img_tensor = model.get_image_features(**image_inputs)
  img_tensor = model(**image_inputs).image_embeds.squeeze(0)
  white_img_array = img_tensor.detach().numpy()

def white_non_white(x, white_img_array):
  if (np.round(x, decimals = 2) == np.round(white_img_array, decimals = 2)).all():
    return 'Blank Image'
  else:
    return 'Valid Image'


#Create a List of Dataframes
df_list = []

for i in tqdm(range(4)):
  df_name = '/content/drive/MyDrive/Office Products Embeddings/Office_Products_tokenized_results_' + str(i+1) + '.parquet'
  df = pd.read_parquet(df_name)
  df['text_embeddings'] = df['tokenized_text']
  df['image_embeddings'] = df['tokenized_image']

  df = df.loc[df['similarity'] > 0.2]
  df = df.reset_index(drop = True)

  for j in range(len(df)):
    # df.loc[j, 'image_type'] = white_non_white(df.loc[i, 'image_embeddings'], white_img_array, df.loc[i, 'image_url'])
    df.loc[j, 'image_type'] = white_non_white(df.loc[i, 'image_embeddings'], white_img_array)

  df_list.append(df)
  print(f"Completed Dataframe {df_name}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
 25%|██▌       | 1/4 [00:30<01:30, 30.12s/it]

Completed Dataframe /content/drive/MyDrive/Office Products Embeddings/Office_Products_tokenized_results_1.parquet


 50%|█████     | 2/4 [01:00<01:00, 30.09s/it]

Completed Dataframe /content/drive/MyDrive/Office Products Embeddings/Office_Products_tokenized_results_2.parquet


 75%|███████▌  | 3/4 [01:30<00:30, 30.38s/it]

Completed Dataframe /content/drive/MyDrive/Office Products Embeddings/Office_Products_tokenized_results_3.parquet


100%|██████████| 4/4 [01:59<00:00, 29.80s/it]

Completed Dataframe /content/drive/MyDrive/Office Products Embeddings/Office_Products_tokenized_results_4.parquet


In [ ]:
#Keep only products that do not have white images
for df in tqdm(df_list):
  df = df.loc[df['image_type'] == 'Valid Image']
  df = df.reset_index(drop = True)

100%|██████████| 4/4 [00:00<00:00, 13.00it/s]


In [1]:
#Get count of total number of Observations remaining
length = 0

for i in df_list:
  length += len(i)

length

In [ ]:
from google.colab import userdata

pinecone_api = "API KEY GOES HERE"

pc = pinecone.Pinecone(api_key=pinecone_api)
index = pc.Index("PINECONE INDEX NAME GOES HERE",
                 host = "HOST URL OF PINECONE INDEX GOES HERE")


In [ ]:
#Check if any product exceeds max upload size limit of Pinecone per vector
import json

for i, row in tqdm(df_list[0].iterrows()):
  batch = []
  total_size = 0
  vector_id = row['parent_asin']

  # Extract vector (assuming it's a numpy array)
  # vector = (row['text_embeddings'] + row['image_embeddings'])
  # vector = np.concatenate((row['text_embeddings'], row['image_embeddings']), axis = 0)
  vector = row['image_embeddings']


  metadata = {
              'parent_asin': row['parent_asin'],
              'title': row['title'],
              'description': row['description'],
              # 'features': row['features'],
              'image_url': row['image_url'],
              'text_image_cosine_similarity': row['similarity'],
              # 'image_type': row['image_type']
          }


  metadata_json = json.dumps(metadata)
  total_size = len(metadata_json.encode('utf-8'))

  if total_size > 40960:
    print(i, row, total_size)

  counter = 0

  #40960 bytes is the max metadata size that a vector can have on Pinecone
  #If size exceeds limit, reduce description length
  while total_size > 40960:
    new_length = int(len(metadata['description'])/2)
    metadata['description'] = row['description'][:new_length]
    metadata_json = json.dumps(metadata)
    total_size = len(metadata_json.encode('utf-8'))
    print(total_size, len(metadata['description']))

8943it [00:00, 11239.36it/s]

6575 parent_asin                                                B00006BBEL
title               HP Premium Plus Photo Paper, High Gloss (50, 8...
description         Product Description, HP's best all-around phot...
image_url           https://m.media-amazon.com/images/I/5153BV-O6T...
tokenized_text      [0.16396813, -0.41250056, 0.12949368, -0.31431...
tokenized_image     [0.77177644, 0.042132437, 0.33375576, -0.35702...
similarity                                                   0.337588
text_embeddings     [0.16396813, -0.41250056, 0.12949368, -0.31431...
image_embeddings    [0.77177644, 0.042132437, 0.33375576, -0.35702...
image_type                                                Valid Image
Name: 6575, dtype: object 82171
41223 40948
20743 20474


55234it [00:04, 12104.93it/s]

52886 parent_asin                                                B001664XLY
title                     Lexmark X2600 All -in-One Printer (12L1685)
description         Overview \t\t\t \t\t\t \t\t \t \t\t \t\t\t \t\...
image_url           https://m.media-amazon.com/images/I/41Q4z-DrGr...
tokenized_text      [0.03120926, -0.2801697, -0.03177153, -0.28221...
tokenized_image     [0.19849995, -0.18025938, 0.39231557, 0.081128...
similarity                                                   0.327669
text_embeddings     [0.03120926, -0.2801697, -0.03177153, -0.28221...
image_embeddings    [0.19849995, -0.18025938, 0.39231557, 0.081128...
image_type                                                Valid Image
Name: 52886, dtype: object 84235
42322 41849
21393 20924


99361it [00:08, 11391.33it/s]


In [ ]:
#Using Multiple CPU Cores to upsertion to Pinecone
import sys
import json


# Function to handle upsertion for each chunk
def upload_chunk_to_pinecone(dataframe_chunk):
    # text_batch = []
    # image_batch = []
    batch = []

    for i, row in tqdm(dataframe_chunk.iterrows()):
        total_size = 0

        # Define unique vector ID (e.g., 'vector_0', 'vector_1', etc.)

        vector_id = row['parent_asin']

        # Extract vector (assuming it's a numpy array)
        # vector = (row['text_embeddings'] + row['image_embeddings'])/2 #Average
        # vector = row['text_embeddings'] + row['image_embeddings'] #Addition
        # vector = np.concatenate((row['text_embeddings'], row['image_embeddings']), axis = 0) #Concatenation
        vector = row['image_embeddings'] #Only Image


        # Create metadata dictionary
        metadata = {
            'parent_asin': row['parent_asin'],
            'title': row['title'],
            'description': row['description'],
            # 'features': row['features'],
            'image_url': row['image_url'],
            'text_image_cosine_similarity': row['similarity'],
            # 'image_type': row['image_type']
        }


        max_size = 40960

        metadata_json = json.dumps(metadata)
        total_size += len(metadata_json.encode('utf-8'))

        #40960 bytes is the max metadata size that a vector can have on Pinecone
        #If size exceeds limit, reduce description length
        while total_size > 40960:
          new_length = int(len(metadata['description'])/2)
          metadata['description'] = row['description'][:new_length]
          metadata_json = json.dumps(metadata)
          total_size = len(metadata_json.encode('utf-8'))
        batch.append((vector_id, vector, metadata))

        # Upload batch in chunks of 500 vectors ---> Switch to 200 when using concat
        if len(batch) == 500:
          index.upsert(vectors=batch)
          batch = []

    # Upload any remaining vectors in the batch
    if batch:
        index.upsert(vectors=batch)


# Function to perform multiprocessing across all dataframes
def parallel_upload(dataframes, num_processes=2):
    with mp.Pool(num_processes) as pool:
        pool.map(upload_chunk_to_pinecone, dataframes)


parallel_upload(df_list, num_processes=4)


89673it [05:10, 288.39it/s]
99361it [05:36, 295.23it/s]
99336it [05:43, 289.52it/s]
99364it [05:43, 289.13it/s]
